In [1]:
import pandas as pd
import requests
import json
import numpy as np
import time

In [ ]:
#Function to query anilist.co for a page from thier anime database, excluding explicit content and including only anime with the FINISHED status.
#It also creates a useable data frame for the query.
def anilist_query(i) -> pd.DataFrame:
  query= '''
  query Query($type: MediaType, $status: MediaStatus, $page: Int, $isAdult: Boolean) {
    Page(page: $page) {
      pageInfo {
        currentPage
        hasNextPage
      }
      media(type: $type, status: $status, isAdult: $isAdult) {
        id
        title {
          english
          romaji
        }
        type
        format
        status
        seasonYear
        source
        genres
        popularity
        tags {
          name
        }
      }
    }
  }
  ''' 
  variables={
    "type" : 'ANIME',
    "status": 'FINISHED',
    "page" : i,
    "isAdult" : False
  }
  response = requests.post(url, json={'query': query,'variables': variables}) # Make the HTTP Api request
  json = response.json()
  df = pd.json_normalize(json['data']) #parse json for data column
  return df

In [ ]:
# Function to loop all pages and append to data frame
def anilist_compile(i) -> pd.DataFrame:
    result = pd.DataFrame() #Create empty data frame.
    while True:
        df = anilist_query(i)
        if df['Page.pageInfo.hasNextPage'][0] == np.True_: #test for if there is another page to query
            df_1 = pd.json_normalize(df['Page.media']).T #flaten db for Page.media and transform
            df_2 = pd.json_normalize(df_1[0])  #flaten db for concatinate
            result = pd.concat([result,df_2], axis=0) #concatinate for result
            i += 1
            time.sleep(3) #limit request rate to complie with anilist
        else:
            df_1 = pd.json_normalize(df['Page.media']).T #flaten db for Page.media and transform
            df_2 = pd.json_normalize(df_1[0]) #flaten db for concatinate
            result = pd.concat([result,df_2], axis=0) #concatinate for result
            break
    return result

In [5]:
url = 'https://graphql.anilist.co'

In [ ]:
anilist = anilist_compile(1) #compiling from page 1 will ensure all titles are included

C:\Users\Leolion023\AppData\Local\Temp\ipykernel_18372\71023842.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result,df_2], axis=0) #concatinate for result
C:\Users\Leolion023\AppData\Local\Temp\ipykernel_18372\71023842.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result,df_2], axis=0) #concatinate for result
C:\Users\Leolion023\AppData\Local\Temp\ipykernel_18372\71023842.py:9: FutureWarning: The behavior of DataFrame concatenation with empty 

In [12]:
anilist

,id,type,format,status,seasonYear,source,genres,popularity,tags,title.english,title.romaji
0,1,ANIME,TV,FINISHED,1998.0,ORIGINAL,"[Action, Adventure, Drama, Sci-Fi]",380694,"[{'name': 'Space'}, {'name': 'Crime'}, {'name'...",Cowboy Bebop,Cowboy Bebop
1,5,ANIME,MOVIE,FINISHED,2001.0,ORIGINAL,"[Action, Drama, Mystery, Sci-Fi]",70431,"[{'name': 'Terrorism'}, {'name': 'Primarily Ad...",Cowboy Bebop: The Movie - Knockin' on Heaven's...,Cowboy Bebop: Tengoku no Tobira
2,6,ANIME,TV,FINISHED,1998.0,MANGA,"[Action, Adventure, Comedy, Drama, Sci-Fi]",136780,"[{'name': 'Guns'}, {'name': 'Fugitive'}, {'nam...",Trigun,TRIGUN
3,7,ANIME,TV,FINISHED,2002.0,ORIGINAL,"[Action, Drama, Mystery, Supernatural]",18663,"[{'name': 'Conspiracy'}, {'name': 'Police'}, {...",Witch Hunter ROBIN,Witch Hunter ROBIN
4,8,ANIME,TV,FINISHED,2004.0,MANGA,"[Adventure, Fantasy, Supernatural]",2614,"[{'name': 'Shounen'}, {'name': 'Spearplay'}, {...",Beet the Vandel Buster,Bouken Ou Beet
...,...,...,...,...,...,...,...,...,...,...,...
36,186761,ANIME,MOVIE,FINISHED,NaN,OTHER,"[Comedy, Drama]",3,[],None,Ban Ye Ji Jiao
37,186762,ANIME,MOVIE,FINISHED,NaN,OTHER,[Comedy],4,[],None,Cao Chong Cheng Xiang
38,186763,ANIME,MOVIE,FINISHED,NaN,OTHER,"[Drama, Fantasy]",4,[{'name': 'Dragons'}],None,Diao Long Ji
39,186764,ANIME,MOVIE,FINISHED,2024.0,None,[Fantasy],138,"[{'name': 'Virtual World'}, {'name': 'Female P...",None,SINoALICE: Ichiban Saigo no Monogatari


In [13]:
anilist.to_csv("complete_anilist.csv",index=False)

In [ ]:
# Function to query anilist.co for a defined user's anime media list with the completed status, and transform it into a useable data frame.
def my_anilist(user_name) -> pd.DataFrame:
    query = '''
    query Query($userName: String, $type: MediaType, $status: MediaListStatus) {
    MediaListCollection(userName: $userName, type: $type, status: $status) {
        lists {
        entries {
            id
            mediaId
            media {
            title {
                english
                romaji
            }
            format
            }
        }
        name
        }
    }
    }
        '''
    variables={
        "type" : 'ANIME',
        "status": 'COMPLETED',
        "userName" : user_name,
    }
    response = requests.post(url, json={'query': query,'variables': variables}) # Make the HTTP Api request
    json = response.json()
    df = pd.json_normalize(json['data']) #parse json for data column
    df = pd.json_normalize(df['MediaListCollection.lists']) 
    df =pd.json_normalize(df[0])
    df = pd.json_normalize(df['entries']).T #transform data frame to be readable
    df = pd.json_normalize(df[0])
    df = df[(df['media.format'] != "MOVIE") & (df['media.format'] != "SPECIAL") & (df['media.format'] != "MUSIC") & (df['media.format'] != "TV_SHORT")] #remove unnecesary formats
    df['media.title.english'] = df['media.title.english'].fillna(df['media.title.romaji'])  #clean english title
    df = df.drop(['media.title.romaji','id','media.format'],axis=1) #drop romaji title, userid, and the format of the media
    return df

In [ ]:
my_list = my_anilist("leolion023") #change username to import your personal list
my_list

,mediaId,media.title.english
1,21711,91 Days
2,9776,A-Channel
3,8101,Hen Zemi
4,20785,Absolute Duo
5,21823,ACCA: 13-Territory Inspection Dept.
...,...,...
1398,112153,Pokémon Journeys: The Series
1399,103301,ENDRO!
1400,103638,Case File nº221: Kabukicho
1401,103222,Magical Girl Spec-Ops Asuka


In [13]:
my_list.to_csv('my_anilist.csv', index=False)